In [3]:
import serial
import time

# 创建一个串行实例。
bluetoothSerial = serial.Serial('COM6', baudrate=9600, timeout=1)
counter=1
try:
    while True:
        if counter==1:
            time.sleep(1)
        if bluetoothSerial.inWaiting() > 0:
            data = bluetoothSerial.readline().decode('utf-8').rstrip()
            print("Received:", data)
        time.sleep(0.1)
        counter-=1
except KeyboardInterrupt:
    bluetoothSerial.close()  # 确保端口被正确关闭


SerialException: could not open port 'COM6': PermissionError(13, '拒绝访问。', None, 5)

In [1]:
import serial
import time

signal_history = []
loop_run = False
max_history_length = 15
required_stable_count = 4

def read_signal(bluetoothSerial):
    # 忽略第一个数据包
    initial_line = bluetoothSerial.readline().decode('utf-8').rstrip()
    # print("忽略第一个数据包:", initial_line)
    
    while True:
        if bluetoothSerial.inWaiting() > 0:
            # 读取数据包
            data_line = bluetoothSerial.readline().decode('utf-8').rstrip()
            data_from_com = data_line.split(',')
            # 确保接收到的数据包长度是7（完整的数据包）
            if len(data_from_com) == 7:
                try:
                    mode = int(float(data_from_com[0]))
                    X_acc = 1 if float(data_from_com[1]) > 0.3 else -1 if float(data_from_com[1]) < -0.3 else 0
                    Y_acc = 1 if float(data_from_com[2]) > 0.3 else -1 if float(data_from_com[2]) < -0.3 else 0
                    Z_acc = 1 if float(data_from_com[3]) > 0.5 else -1 if float(data_from_com[3]) < -0.5 else 0
                    gloveRoll = 1 if float(data_from_com[4]) > 10 else -1 if float(data_from_com[4]) < -10 else 0
                    glovePitch = 1 if float(data_from_com[5]) > 10 else -1 if float(data_from_com[5]) < -10 else 0
                    gloveYaw = 1 if float(data_from_com[6]) > 10 else -1 if float(data_from_com[6]) < -10 else 0
                    print(data_from_com)
                    return (mode, X_acc, Y_acc, Z_acc, gloveRoll, glovePitch, gloveYaw)
                except ValueError:
                    print("数据格式错误:", data_from_com)
            else:
                print("收到不完整的数据包:", data_from_com)
        time.sleep(0.1)


def update_signal_history(signal_history, new_signal, max_history_length):
    # 在列表的开始位置插入新的信号
    signal_history.insert(0, tuple(new_signal))
    
    # 如果历史记录的长度超过了设定的最大长度，则移除最旧的信号（列表末尾的元素）
    if len(signal_history) > max_history_length:
        signal_history.pop()

def find_stable_signal(signal_history, required_stable_count):
    if len(signal_history) < required_stable_count:
        return None

    # 检查每个可能的信号模式是否在历史中稳定
    for signal_pattern in set(signal_history):
        if signal_history.count(signal_pattern) >= required_stable_count:
            return signal_pattern
    return None

### main starts here
if __name__ == '__main__':
    bluetoothSerial = serial.Serial('COM6', baudrate=9600, timeout=1)
    time.sleep(0.5)
    loop_run = True
    while loop_run:
        
        current_signal = read_signal(bluetoothSerial)  # 模拟读取当前信号
        # print(current_signal)
        update_signal_history(signal_history, current_signal, max_history_length)
        stable_signal = find_stable_signal(signal_history, required_stable_count)
        # print(stable_signal)

['5.00', '0.19', '0.04', '0.53', '-5.88', '-3.95', '-10.48']
['5.00', '0.35', '0.05', '0.67', '-8.57', '-6.60', '-11.39']
['5.00', '0.25', '-0.07', '0.59', '-8.83', '-6.64', '-11.08']
['5.00', '0.24', '0.01', '0.58', '-6.77', '-5.14', '-10.27']
['5.00', '0.23', '0.06', '0.56', '-7.27', '-3.86', '-10.11']
['5.00', '0.09', '-0.00', '0.53', '-6.92', '-1.57', '-9.82']
['5.00', '0.11', '-0.02', '0.51', '-7.36', '-2.03', '-10.14']
['5.00', '0.10', '-0.07', '0.53', '-7.39', '-3.18', '-9.67']
['5.00', '0.24', '0.11', '0.56', '-7.22', '-6.10', '-10.04']
['5.00', '0.31', '-0.04', '0.62', '-9.22', '-7.64', '-10.38']
收到不完整的数据包: ['5.00.02', '0.60', '-8.88', '-7.13', '-10.32']
['5.00', '0.27', '-0.02', '0.60', '-8.21', '-6.10', '-10.09']
['5.00', '0.24', '0.09', '0.58', '-7.25', '-4.10', '-10.26']
['5.00', '0.11', '-0.12', '0.54', '-8.36', '-2.19', '-10.34']
['5.00', '0.07', '-0.06', '0.50', '-7.88', '-0.59', '-10.08']
['5.00', '0.11', '-0.05', '0.52', '-8.23', '-1.12', '-10.25']
['5.00', '0.05', '-

KeyboardInterrupt: 